In [2]:
# Install PyDrive
!pip install PyDrive
!pip install pandas
import pandas as pd
#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the Shareable link
#Ex BIG : https://drive.google.com/file/d/1aWNJjMp9Ks-ponG8MLpHz6eRYS-gzA4i/view?usp=sharing
#https://drive.google.com/file/d/143Je73S7FHWusXN2n-gYq86BWtFwufu8/view?usp=sharing

#ORIGINALE
#https://drive.google.com/file/d/1QEaoJiiCvakAo-zf79gjhK0Aw9GbsdvG/view?usp=sharing
downloaded = drive.CreateFile({'id':"143Je73S7FHWusXN2n-gYq86BWtFwufu8"})   
downloaded.GetContentFile('ds_train1.csv')       



In [3]:
import pandas as pd

df = pd.read_csv("ds_train1.csv",encoding  = "UTF-8")
df_not_na = df[~(df['abstract_cleaned'].isna())]


In [4]:
abstract_0 = df_not_na['abstract_cleaned']
abstract = abstract_0.apply(lambda x: x.split(","))


In [5]:
df_not_na['abstract_cleaned'] = abstract
df_not_na['Keywords'] = df_not_na['Keywords'].apply(lambda x: x.split(","))
df = df_not_na.explode('Keywords')
df[:10]

,year,abstract,Keywords,text,abstract_spacy,abstract_cleaned
0,2014,"visual,analytics,inherently,collaboration,curr...",analytic_provenance,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",applied_machine_learning,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",user_interaction,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",flow_visualization,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",adolescent,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",adult,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",computer_graphics,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",computer_interface,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",decision_making,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."
0,2014,"visual,analytics,inherently,collaboration,curr...",image_processing,"visual,analytics,inherently,collaboration,curr...",analytics collaboration analytics interaction ...,"[analytic_provenance, applied_machine_learning..."


In [6]:
df_mf = df.groupby('Keywords').filter(lambda x: x['Keywords'].count()>10)


In [16]:
keywords = df['Keywords'].value_counts()[:30]
keywords

learning_systems                   8587
artificial_intelligence            5873
machine_learning                   5155
learning_algorithms                3916
support_vector_machines            2854
classification_(of_information)    2514
data_mining                        1927
neural_networks                    1625
feature_extraction                 1532
support_vector_machine             1399
extreme_learning_machine           1389
decision_trees                     1289
machine_learning_techniques        1223
deep_learning                      1131
knowledge_acquisition              1020
regression_analysis                1010
artificial_neural_network           925
machine_learning_methods            832
procedure                           819
supervised_learning                 801
pattern_recognition                 793
computer_simulation                 740
male                                728
semantics                           684
classifier                          682


In [26]:
df_filtered = df.groupby('Keywords').filter(lambda x: x['Keywords'].count()>=1532)

In [58]:
len(df_filtered)


33983

In [28]:
x = df_filtered['abstract_cleaned']
y = df_filtered['Keywords'].tolist()

In [47]:
print(type(x))

<class 'pandas.core.series.Series'>


### preprocessing

### import embedding models

In [29]:
from gensim.models import KeyedVectors 
embeddings = KeyedVectors.load_word2vec_format('ft_model_cb.vec', binary=True)


### text to embeddings

#### tokenize and pad

In [55]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

max_length = 100
t = Tokenizer()
t.fit_on_texts(x)
vocab_size = len(t.word_index) + 1
emb_size=embeddings.vector_size

encoded_docs = t.texts_to_sequences(x)
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.3, random_state=42, stratify=y)

In [64]:
type(padded_docs)

numpy.ndarray

#### create embedding weights matrix

In [31]:
import numpy as np
import tensorflow as tf
tf.random.set_seed(1)

# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, emb_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        if word in embeddings.vocab:
            weight_matrix[i] = embedding[word]
    return weight_matrix

### create embedding features

In [57]:
len(list(set(y)))

9

### LSTM classification

In [33]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, LSTM, Bidirectional, GRU, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# get vectors in the right order
embedding_vectors = get_weight_matrix(embeddings, t.word_index)
print(embedding_vectors.shape)
e = Embedding(vocab_size, emb_size, weights=[embedding_vectors], input_length=max_length, trainable=False)

#params
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
n_labels = len(list(set(y)))
# create the model
def baseline_model(optimizer=opt):
    model = Sequential()
    model.add(e)
    #layer che considera le parole in sequenza 50 neuroni
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(n_labels, activation='softmax'))
    #loss misura l'errore ed è categorica perchè facciamo classificazione
    #in base alla loss la rete cambia iterativamente le feature e sceglie quelle che minimizzano la loss n.iterazioni = epoche
    #tante feature fanno decadere l'efficienza
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    print(model.summary())
    return model

model = KerasClassifier(build_fn=baseline_model)
# batch_size = [32, 64,128]
# epochs = [5, 10, 20]
# optimizer = ['Adadelta', 'Nadam']
batch_size = [64] #numero di frasi prese alla volta e modificano i pesi 
epochs = [5]
optimizer = ['Adam']#algoritmo che calcola l'errore
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='f1_weighted', verbose=2)
print('***CROSS VALIDATED GRID SEARCH***')
%time grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

(54531, 10)
***CROSS VALIDATED GRID SEARCH***
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 11.7min finished


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 10)           545310    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100, 50)           12200     
_________________________________________________________________
lstm_7 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 459       
Total params: 578,169
Trainable params: 32,859
Non-trainable params: 545,310
_________________________________________________________________
None
Epoch 1/10
372/372 [==============================] - 40s 108ms/step - loss: 2.0488 - accuracy: 0.2524
Epoch 2/10
372/372 [==============================] - 41s 111ms/step - loss: 2.0368 - accuracy: 0.2527
Epoch 3/10
372/372 [==========

### predict test set

In [78]:
pred

array([0.17380586, 0.07119283, 0.05620811, 0.04656512, 0.10976699,
       0.24659415, 0.16123617, 0.04837783, 0.08625301], dtype=float32)

In [48]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
#results on the test set for the best model
best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred,average='micro')
print('***RESULTS ON TEST SET***')
print("accuracy_score", accuracy)
print("f1_score", f1)
print('\n')
print(classification_report(y_test, y_pred))

#macro la media della f1 tra una classe e l'altra

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
***RESULTS ON TEST SET***
accuracy_score 0.25267287886218737
f1_score 0.25267287886218737


                                 precision    recall  f1-score   support

        artificial_intelligence       0.00      0.00      0.00      1762
classification_(of_information)       0.00      0.00      0.00       754
                    data_mining       0.00      0.00      0.00       578
             feature_extraction       0.00      0.00      0.00       460
            learning_algorithms       0.00      0.00      0.00      1175
               learning_systems       0.25      1.00      0.40      2576
               machine_learning       0.00      0.00      0.00

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
probability_class_1 = best_model.predict_proba(x_test[0])[:, 1]
probability_class_2 = best_model.predict_proba(x_test[0])[:, 2]
probability_class_3 = best_model.predict_proba(x_test[0])[:, 3]
probability_class_4 = best_model.predict_proba(x_test[0])[:, 4]
probability_class_5 = best_model.predict_proba(x_test[0])[:, 5]
probability_class_6 = best_model.predict_proba(x_test[0])[:, 6]
probability_class_7 = best_model.predict_proba(x_test[0])[:, 7]
probability_class_8 = best_model.predict_proba(x_test[0])[:, 8]

probability_class_1

array([0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.1022577 , 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225769,
       0.10225769, 0.10225769, 0.10225769, 0.10225769, 0.10225

In [91]:
probability_class_2

array([0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363224, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363222,
       0.09363222, 0.09363222, 0.09363222, 0.09363222, 0.09363

In [92]:
probability_class_3

array([0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996257,
       0.08996256, 0.08996257, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996256,
       0.08996256, 0.08996256, 0.08996256, 0.08996256, 0.08996

In [93]:
probability_class_4

array([0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670273,
       0.11670272, 0.11670274, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670272,
       0.11670272, 0.11670272, 0.11670272, 0.11670272, 0.11670

### More results with more resources

<img src='img/48workers.PNG'>